In [36]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, accuracy_score


In [37]:

# Configuration
MODEL_PATH = "./sentiment_model"
TEST_DATA_PATH = "./twitter_training.csv"
BATCH_SIZE = 32
MAX_LEN = 128


In [38]:
import pandas as pd

# Load and inspect the CSV file
file_path = "./twitter_training.csv"
try:
    df = pd.read_csv(file_path)
    print("Dataset columns:", df.columns.tolist())
    print("First few rows:\n", df.head())
except Exception as e:
    print(f"Error loading dataset: {e}")


Dataset columns: ['2401', 'Borderlands', 'Positive', 'im getting on borderlands and i will murder you all ,']
First few rows:
    2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     


In [39]:
# Load the trained model and tokenizer
print("Loading model and tokenizer...")
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH)
    tokenizer = DistilBertTokenizer.from_pretrained(MODEL_PATH)
    model.to(device)
    model.eval()
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    raise



Loading model and tokenizer...
Model and tokenizer loaded successfully.


In [40]:

# Define Dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row["text"]
        label = row["label"]

        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }


In [41]:
import pandas as pd

# Load and preprocess the test data
def load_test_data(file_path, sample_size):
    print("Loading test data...")
    df = pd.read_csv(file_path)
    
    # Rename columns to match required format
    df = df.rename(columns={
        df.columns[3]: "text",  # Assuming the 4th column is the text
        df.columns[2]: "label"  # Assuming the 3rd column is the label
    })
    
    # Ensure correct column names exist
    required_columns = ["text", "label"]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Dataset is missing required columns: {required_columns}")
    
    # Drop rows with missing values in critical columns
    df = df.dropna(subset=required_columns)
    
    # Map labels to integers
    label_mapping = {
        "Positive": 2,  # Most frequent in training
        "Neutral": 1,
        "Negative": 0,
        "Irrelevant": 3  # Least frequent in training
    }
    df["label"] = df["label"].map(label_mapping)
    
    # Validate label mapping
    if df["label"].isnull().any():
        raise ValueError("Found unmapped labels in the dataset.")
    
    df_sampled = df.head(sample_size)
    
    print("Test data loaded and processed successfully.")
    return df_sampled

# Usage
try:
    TEST_DATA_PATH = "./twitter_training.csv"
    test_data = load_test_data(TEST_DATA_PATH, sample_size=1000)
except Exception as e:
    print(f"Error loading test data: {e}")
    raise

Loading test data...
Test data loaded and processed successfully.


In [42]:

# Create DataLoader
test_dataset = SentimentDataset(test_data, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Evaluate the model
def evaluate_model(model, data_loader):
    print("Evaluating model...")
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Evaluation complete.")
    return all_labels, all_preds


In [43]:

# Generate predictions and performance metrics
try:
    true_labels, predictions = evaluate_model(model, test_loader)
    print("Generating performance report...")
    print(classification_report(true_labels, predictions, target_names=["Positive", "Neutral", "Negative", "Irrelevant"]))
    print(f"Accuracy: {accuracy_score(true_labels, predictions):.2f}")
except Exception as e:
    print(f"Error during evaluation: {e}")
    raise

Evaluating model...
Evaluation complete.
Generating performance report...
              precision    recall  f1-score   support

    Positive       0.79      0.81      0.80       195
     Neutral       0.69      0.71      0.70       279
    Negative       0.83      0.83      0.83       424
  Irrelevant       0.69      0.64      0.66       102

    accuracy                           0.77      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.77      0.77      0.77      1000

Accuracy: 0.77
